# Distinguishing muffins and Chihuahuas with NNs

The aim of this project is to train a neural network for the binary classification of muffins and Chihuahuas based on the images contained in the relative [Kaggle dataset](https://www.kaggle.com/datasets/samuelcortinhas/muffin-vs-chihuahua-image-classification).

We import the Yelp dataset from Kaggle, using a token.

In [3]:
import os
os.environ['KAGGLE_USERNAME'] = "luciaannamellini"
os.environ['KAGGLE_KEY'] = "c209fcf223ecdd6be8fd373196354f4b"
!kaggle datasets download -d samuelcortinhas/muffin-vs-chihuahua-image-classification


  0%|          | 0.00/474M [00:00<?, ?B/s]
  0%|          | 1.00M/474M [00:00<05:04, 1.63MB/s]
  1%|          | 3.00M/474M [00:00<01:44, 4.71MB/s]
  1%|▏         | 6.00M/474M [00:00<00:50, 9.77MB/s]
  2%|▏         | 11.0M/474M [00:01<00:27, 17.9MB/s]
  3%|▎         | 16.0M/474M [00:01<00:20, 23.8MB/s]
  4%|▍         | 21.0M/474M [00:01<00:20, 23.4MB/s]
  5%|▌         | 25.0M/474M [00:01<00:17, 26.9MB/s]
  6%|▋         | 30.0M/474M [00:01<00:15, 29.4MB/s]
  7%|▋         | 35.0M/474M [00:01<00:13, 33.4MB/s]
  8%|▊         | 40.0M/474M [00:01<00:12, 35.8MB/s]
  9%|▉         | 44.0M/474M [00:01<00:12, 35.9MB/s]
 10%|█         | 48.0M/474M [00:02<00:11, 37.4MB/s]
 11%|█         | 52.0M/474M [00:02<00:11, 37.0MB/s]
 12%|█▏        | 57.0M/474M [00:02<00:11, 38.5MB/s]
 13%|█▎        | 61.0M/474M [00:02<00:11, 36.4MB/s]
 14%|█▎        | 65.0M/474M [00:02<00:11, 37.3MB/s]
 15%|█▍        | 69.0M/474M [00:02<00:12, 35.3MB/s]
 16%|█▌        | 74.0M/474M [00:02<00:10, 38.2MB/s]
 17%|█▋        | 79.